In [26]:
from ner import *

In [2]:
# get data and preprocess named_entities
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
df['named_entities_preprocessed'] = df['named_entities'].apply(clean_named_entities)
df = join_split_entities(df)
df['named_entities_preprocessed'] = df['named_entities_preprocessed'].apply(unify_entities)

In [3]:
all_entities = get_ne(df, ['LOC', 'ORG', 'PER'])

In [33]:
show_top_x_ne(df, 0.03, ['LOC', 'ORG', 'PER']) # top NE in the whole corpus

россия 9986 0.81
организация объединённых наций 4520 0.37
москва 3854 0.31
сша 3052 0.25
лавров 1939 0.16
сирия 1865 0.15
путин 1689 0.14
украина 1680 0.14
европейский союз 1663 0.13
организация по безопасности и сотрудничеству в европе 1414 0.11
европа 1409 0.11
ближний восток 1274 0.1
совет безопасности оон 1162 0.09
вашингтон 976 0.08
афганистан 917 0.07
содружество независимых государств 916 0.07
нато 860 0.07
ирак 801 0.06
медведев 740 0.06
киев 734 0.06
китайская народная республика 728 0.06
иран 686 0.06
шанхайская организация сотрудничества 647 0.05
генеральная ассамблея оон 643 0.05
франция 625 0.05
северный африка 624 0.05
германия 622 0.05
иг 618 0.05
ливия 594 0.05
ссср 579 0.05
совет европы 561 0.05
организация договора о коллективной безопасности 546 0.04
санкт-петербург 542 0.04
великобритания 531 0.04
нью-йорк 525 0.04
азиатско-тихоокеанский регион 514 0.04
африка 513 0.04
турция 510 0.04
запад 492 0.04
женева 463 0.04
дамаск 453 0.04
сочи 444 0.04
совет безопасность 44

{'россия': 9986,
 'организация объединённых наций': 4520,
 'москва': 3854,
 'сша': 3052,
 'лавров': 1939,
 'сирия': 1865,
 'путин': 1689,
 'украина': 1680,
 'европейский союз': 1663,
 'организация по безопасности и сотрудничеству в европе': 1414,
 'европа': 1409,
 'ближний восток': 1274,
 'совет безопасности оон': 1162,
 'вашингтон': 976,
 'афганистан': 917,
 'содружество независимых государств': 916,
 'нато': 860,
 'ирак': 801,
 'медведев': 740,
 'киев': 734,
 'китайская народная республика': 728,
 'иран': 686,
 'шанхайская организация сотрудничества': 647,
 'генеральная ассамблея оон': 643,
 'франция': 625,
 'северный африка': 624,
 'германия': 622,
 'иг': 618,
 'ливия': 594,
 'ссср': 579,
 'совет европы': 561,
 'организация договора о коллективной безопасности': 546,
 'санкт-петербург': 542,
 'великобритания': 531,
 'нью-йорк': 525,
 'азиатско-тихоокеанский регион': 514,
 'африка': 513,
 'турция': 510,
 'запад': 492,
 'женева': 463,
 'дамаск': 453,
 'сочи': 444,
 'совет безопасность

In [5]:
sorted_dfs = sort_by_official_dates(df) 

In [32]:
show_top_x_ne(sorted_dfs[0], 0.03, ['LOC', 'ORG', 'PER']) # < 2008

россия 1396 0.88
организация объединённых наций 716 0.45
москва 588 0.37
путин 383 0.24
европейский союз 358 0.23
сша 312 0.2
европа 301 0.19
содружество независимых государств 238 0.15
ирак 224 0.14
совет безопасности оон 207 0.13
организация по безопасности и сотрудничеству в европе 204 0.13
ближний восток 176 0.11
лавров 163 0.1
совет европы 153 0.1
санкт-петербург 150 0.09
афганистан 146 0.09
нато 142 0.09
иран 133 0.08
китайская народная республика 132 0.08
республика косово 129 0.08
ссср 129 0.08
израиль 110 0.07
совет безопасность 104 0.07
украина 99 0.06
азиатско-тихоокеанский регион 98 0.06
генеральная ассамблея оон 96 0.06
шанхайская организация сотрудничества 85 0.05
грузия 84 0.05
азия 84 0.05
организация договора о коллективной безопасности 84 0.05
индия 82 0.05
нью-йорк 81 0.05
франция 79 0.05
группа восемь 79 0.05
совет россия — нато 77 0.05
центральная азия 73 0.05
германия 72 0.05
ливан 70 0.04
государство палестина 70 0.04
африка 70 0.04
федеральное собрание 69 0.04
б

{'россия': 1396,
 'организация объединённых наций': 716,
 'москва': 588,
 'путин': 383,
 'европейский союз': 358,
 'сша': 312,
 'европа': 301,
 'содружество независимых государств': 238,
 'ирак': 224,
 'совет безопасности оон': 207,
 'организация по безопасности и сотрудничеству в европе': 204,
 'ближний восток': 176,
 'лавров': 163,
 'совет европы': 153,
 'санкт-петербург': 150,
 'афганистан': 146,
 'нато': 142,
 'иран': 133,
 'китайская народная республика': 132,
 'республика косово': 129,
 'ссср': 129,
 'израиль': 110,
 'совет безопасность': 104,
 'украина': 99,
 'азиатско-тихоокеанский регион': 98,
 'генеральная ассамблея оон': 96,
 'шанхайская организация сотрудничества': 85,
 'грузия': 84,
 'азия': 84,
 'организация договора о коллективной безопасности': 84,
 'индия': 82,
 'нью-йорк': 81,
 'франция': 79,
 'группа восемь': 79,
 'совет россия — нато': 77,
 'центральная азия': 73,
 'германия': 72,
 'ливан': 70,
 'государство палестина': 70,
 'африка': 70,
 'федеральное собрание': 69

In [34]:
show_top_x_ne(sorted_dfs[1], 0.03, ['LOC', 'ORG', 'PER']) # 2008 - 2013

россия 2692 0.85
организация объединённых наций 1022 0.32
москва 975 0.31
сша 620 0.2
медведев 602 0.19
лавров 494 0.16
европейский союз 442 0.14
организация по безопасности и сотрудничеству в европе 359 0.11
совет безопасности оон 333 0.11
европа 319 0.1
афганистан 306 0.1
ближний восток 294 0.09
содружество независимых государств 275 0.09
грузия 214 0.07
сирия 209 0.07
нато 207 0.07
южная осетия 196 0.06
шанхайская организация сотрудничества 188 0.06
китайская народная республика 186 0.06
украина 173 0.05
иран 170 0.05
организация договора о коллективной безопасности 169 0.05
путин 159 0.05
германия 154 0.05
франция 153 0.05
азиатско-тихоокеанский регион 153 0.05
вашингтон 146 0.05
япония 146 0.05
совет россия — нато 146 0.05
абхазия 145 0.05
ссср 144 0.05
белоруссия 144 0.05
нью-йорк 144 0.05
совет безопасность 142 0.04
генеральная ассамблея оон 139 0.04
казахстан 137 0.04
совет европы 136 0.04
израиль 131 0.04
лига арабских государств 124 0.04
санкт-петербург 124 0.04
северный афри

{'россия': 2692,
 'организация объединённых наций': 1022,
 'москва': 975,
 'сша': 620,
 'медведев': 602,
 'лавров': 494,
 'европейский союз': 442,
 'организация по безопасности и сотрудничеству в европе': 359,
 'совет безопасности оон': 333,
 'европа': 319,
 'афганистан': 306,
 'ближний восток': 294,
 'содружество независимых государств': 275,
 'грузия': 214,
 'сирия': 209,
 'нато': 207,
 'южная осетия': 196,
 'шанхайская организация сотрудничества': 188,
 'китайская народная республика': 186,
 'украина': 173,
 'иран': 170,
 'организация договора о коллективной безопасности': 169,
 'путин': 159,
 'германия': 154,
 'франция': 153,
 'азиатско-тихоокеанский регион': 153,
 'вашингтон': 146,
 'япония': 146,
 'совет россия — нато': 146,
 'абхазия': 145,
 'ссср': 144,
 'белоруссия': 144,
 'нью-йорк': 144,
 'совет безопасность': 142,
 'генеральная ассамблея оон': 139,
 'казахстан': 137,
 'совет европы': 136,
 'израиль': 131,
 'лига арабских государств': 124,
 'санкт-петербург': 124,
 'северный

In [35]:
show_top_x_ne(sorted_dfs[2], 0.03, ['LOC', 'ORG', 'PER']) # 2013 - 2016

россия 2536 0.76
организация объединённых наций 1226 0.37
москва 1044 0.31
сша 805 0.24
сирия 796 0.24
украина 787 0.24
лавров 523 0.16
путин 512 0.15
ближний восток 442 0.13
европейский союз 433 0.13
организация по безопасности и сотрудничеству в европе 412 0.12
европа 374 0.11
киев 365 0.11
иг 335 0.1
северный африка 297 0.09
совет безопасности оон 293 0.09
ирак 257 0.08
вашингтон 223 0.07
ливия 222 0.07
нато 219 0.07
афганистан 218 0.07
дамаск 192 0.06
китайская народная республика 182 0.05
фронт ан-нусра 173 0.05
содружество независимых государств 171 0.05
женева 167 0.05
германия 165 0.05
франция 161 0.05
турция 157 0.05
шанхайская организация сотрудничества 157 0.05
генеральная ассамблея оон 153 0.05
запад 143 0.04
африка 140 0.04
брикс 134 0.04
евразийский экономический союз 132 0.04
азиатско-тихоокеанский регион 130 0.04
брюссель 127 0.04
совет европы 124 0.04
алеппо 123 0.04
крым 123 0.04
йемен 123 0.04
организация договора о коллективной безопасности 118 0.04
ссср 116 0.03
до

{'россия': 2536,
 'организация объединённых наций': 1226,
 'москва': 1044,
 'сша': 805,
 'сирия': 796,
 'украина': 787,
 'лавров': 523,
 'путин': 512,
 'ближний восток': 442,
 'европейский союз': 433,
 'организация по безопасности и сотрудничеству в европе': 412,
 'европа': 374,
 'киев': 365,
 'иг': 335,
 'северный африка': 297,
 'совет безопасности оон': 293,
 'ирак': 257,
 'вашингтон': 223,
 'ливия': 222,
 'нато': 219,
 'афганистан': 218,
 'дамаск': 192,
 'китайская народная республика': 182,
 'фронт ан-нусра': 173,
 'содружество независимых государств': 171,
 'женева': 167,
 'германия': 165,
 'франция': 161,
 'турция': 157,
 'шанхайская организация сотрудничества': 157,
 'генеральная ассамблея оон': 153,
 'запад': 143,
 'африка': 140,
 'брикс': 134,
 'евразийский экономический союз': 132,
 'азиатско-тихоокеанский регион': 130,
 'брюссель': 127,
 'совет европы': 124,
 'алеппо': 123,
 'крым': 123,
 'йемен': 123,
 'организация договора о коллективной безопасности': 118,
 'ссср': 116,
 

In [36]:
show_top_x_ne(sorted_dfs[3], 0.03, ['LOC', 'ORG', 'PER']) # > 2016

россия 3362 0.79
организация объединённых наций 1556 0.36
сша 1315 0.31
москва 1247 0.29
сирия 816 0.19
лавров 759 0.18
путин 635 0.15
украина 621 0.15
вашингтон 560 0.13
организация по безопасности и сотрудничеству в европе 439 0.1
европейский союз 430 0.1
европа 415 0.1
ближний восток 362 0.08
совет безопасности оон 329 0.08
киев 300 0.07
нато 292 0.07
иг 283 0.07
сочи 278 0.06
иран 271 0.06
великобритания 263 0.06
генеральная ассамблея оон 255 0.06
ливия 254 0.06
афганистан 247 0.06
франция 232 0.05
содружество независимых государств 232 0.05
германия 231 0.05
ирак 231 0.05
китайская народная республика 228 0.05
запад 226 0.05
турция 226 0.05
евразийский экономический союз 222 0.05
шанхайская организация сотрудничества 217 0.05
африка 205 0.05
дамаск 194 0.05
ссср 190 0.04
северный африка 187 0.04
нью-йорк 185 0.04
организация договора о коллективной безопасности 175 0.04
санкт-петербург 163 0.04
женева 161 0.04
венесуэла 155 0.04
организация по запрещению химического оружия 154 0.0

{'россия': 3362,
 'организация объединённых наций': 1556,
 'сша': 1315,
 'москва': 1247,
 'сирия': 816,
 'лавров': 759,
 'путин': 635,
 'украина': 621,
 'вашингтон': 560,
 'организация по безопасности и сотрудничеству в европе': 439,
 'европейский союз': 430,
 'европа': 415,
 'ближний восток': 362,
 'совет безопасности оон': 329,
 'киев': 300,
 'нато': 292,
 'иг': 283,
 'сочи': 278,
 'иран': 271,
 'великобритания': 263,
 'генеральная ассамблея оон': 255,
 'ливия': 254,
 'афганистан': 247,
 'франция': 232,
 'содружество независимых государств': 232,
 'германия': 231,
 'ирак': 231,
 'китайская народная республика': 228,
 'запад': 226,
 'турция': 226,
 'евразийский экономический союз': 222,
 'шанхайская организация сотрудничества': 217,
 'африка': 205,
 'дамаск': 194,
 'ссср': 190,
 'северный африка': 187,
 'нью-йорк': 185,
 'организация договора о коллективной безопасности': 175,
 'санкт-петербург': 163,
 'женева': 161,
 'венесуэла': 155,
 'организация по запрещению химического оружия': 

In [5]:
for df in sorted_dfs:
    get_stat_by_region(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
